In [4]:
# pip install langchain-groq
from langchain_groq import ChatGroq

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='***********************',  # Replace with your actual Groq API key
    model_name="llama-3.1-8b-instant"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [ ]:
# Load a web page and extract its content
# Ensure you have the langchain_community package installed
# pip install langchain_community

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/lead-site-reliability-engineer-itc/job/R-62782")
page_data = loader.load().pop().page_content
print(page_data)





















Lead Site Reliability Engineer, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm  # this will form a chain of prompt and llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Site Reliability Engineer, ITC',
 'experience': {'required': 7,
  'type': 'years',
  'field': 'software development, operations, or support'},
 'skills': ['Java',
  'JavaScript (React, Angular, etc.) and backend (Node.js) components',
  'Kubernetes',
  'Docker and/or Serverless patterns',
  'No-SQL database like DynamoDb, Cassandra, etc.',
  'RESTful APIs',
  'ServiceNow, Jira, Jenkins, Splunk, New Relic, SignalFx',
  'ITIL or Lean'],
 'description': 'As a site reliability engineer, you will be focused on maximum availability, observability, reliability, security, and performance for Nike Digital Experiences. SREs perform deep problem analysis, detect infrastructure or code defects, define, report, and create observability processes for Key Performance Indicators (KPIs), and work with product delivery teams to provide long term solutions to production issues.'}

In [ ]:
type(json_res)

dict

In [ ]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

ModuleNotFoundError: No module named 'chromadb'

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [ ]:
job

{'role': 'Software Engineer II, AI/ML Platforms',
 'experience': '2+ years professional software development experience',
 'skills': ['Strong Computer Science fundamentals',
  'Programming experience with at least one modern language such as Java, Python, Golang',
  'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
  'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
  'Experience developing and using microservices'],
 'description': 'We are looking for a Software Engineer II to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the intersection of computer systems and machine learning. You will also work collaboratively with the data science and analytics teams to ship scalable, performant and reliable solutions.'}

In [ ]:
job = json_res
job['skills']

['Strong Computer Science fundamentals',
 'Programming experience with at least one modern language such as Java, Python, Golang',
 'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
 'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
 'Experience developing and using microservices']

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI/ML Solutions for Nike's Data Science Services and Platform

Dear Hiring Manager,

I came across the job description for a Software Engineer II, AI/ML Platforms at Nike, and I'm excited to introduce AtliQ, a leading AI & Software Consulting company that can help you design and implement core components of your data science services and platform.

With our expertise in AI, ML, and software development, we can help you scale your platform to serve Nike globally. Our team of experts has a strong foundation in computer science and experience with modern programming languages such as Java, Python, and Golang. We're well-versed in Infrastructure as Code / DevOps practices and technologies like Terraform, and have hands-on experience with Cloud services like AWS and Azure.

Our portfolio showcases our capabilities in developing and deploying scalable, performant, and reliable solutions. I'd like to highlight a few relevant examples:

* Our Machine Learning with Python portfo